In [3]:
#                                                   LIBRARIES

import os

# Image preprocesssing- processing
import cv2
from PIL import Image
import numpy as np

import re

In [4]:
INPUT_FOLDER = "../datasets/animals/data"
OUTPUT_FOLDER = "../datasets/animals/data_resized"

In [5]:
IMG_EXTENSIONS = ['jpg','JPG','png','PNG']

In [6]:
#                                                 FUNCIONES AUXILIARES


#                                       getFilesWithAbsolutePath(path, ext = '')

# Dado un directorio devuelve una lista con el path absoluto de todos los ficheros con extensión ext

def getFilesWithAbsolutePath(path, ext = ''):
    # to store files in a list
    list = []

    if bool(ext):
        ext = '.' + ext

    # dirs=directories
    for (root, folder, files) in os.walk(path):
        for file in files:
            filePath = os.path.abspath(os.path.join(root, file))
            if ext in filePath:
                list.append(filePath)
                #print(filePath)

    return list

#lista = getFilesWithAbsolutePath(mypath, ext='pdf')
#print(len(lista), lista[:3])


#                            getFilesWithAbsolutePathListaExtensiones(path, extList = [''])

# Dado un directorio devuelve una lista con el path absoluto de
# todos los ficheros con extensión en extList

def getFilesWithAbsolutePathListaExtensiones(path):
    # to store files in a list
    l_list = []
    extList = IMG_EXTENSIONS

    if len(extList) == 0:
        extList = ['']

    for ext in extList:
        l_list += getFilesWithAbsolutePath(path, ext)

    return l_list


# lista = getFilesWithAbsolutePathListaExtensiones(mypath, extList = ['epub','pdf'])
# print(len(lista))

#                       FUNCIONES AUXILIARES PARA ORDENAR LOS NOMBRES DE LAS IMÁGENES DE MODO HUMANO
#                                        Modo humano = por orden alfanumerico

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [32]:
def create_CSV(input_folder = INPUT_FOLDER, tag = ""):

    input_folder = os.path.abspath(input_folder) + os.sep
    csvArray = np.empty([len(getFilesWithAbsolutePathListaExtensiones(input_folder)), 2], dtype=object)

    csvOutput = ".." + os.sep + "csv" + os.sep + "imgs_and_labels_list" + tag + '.csv'

    idx_aux = 0

    for animal in os.listdir(input_folder):
        #print(animal)
        animal_imgs = getFilesWithAbsolutePathListaExtensiones(input_folder + animal)
        animal_imgs.sort(key=natural_keys)
        #print(animal_imgs)

        for img in animal_imgs:
            img_to_insert = img.split("2. Animal Recognition")[1]
            csvArray[idx_aux,:] = [img_to_insert, animal]
            idx_aux += 1

    try:
        np.savetxt(csvOutput, csvArray, delimiter=",",fmt='%s')
        print()
        print('CSV salvado: ', csvOutput)
    except:
        print('ERROR ESCRITURA FINAL CSV:', csvOutput)


def resize_img(img_path):
    basewidth = 200
    baseheight = 200
    try:
        img = Image.open(img_path)

        img = img.resize((basewidth,baseheight), Image.ANTIALIAS) # cambiar pilImage.ANTIALIAS a pilImage.Resampling.LANCZOS
        # img = img.resize((basewidth,hsize), pilImage.Resampling.LANCZOS)
        #print(img.size)
    except:
        print('ERROR LECTURA/REESCALADO (Imagen):', img_path)
        img = 0

    return img


def save_resized_imgs(input_folder = INPUT_FOLDER, output_folder = OUTPUT_FOLDER):
    input_folder = os.path.abspath(input_folder) + os.sep
    output_folder = os.path.abspath(output_folder) + os.sep

    csvArray = np.empty([len(getFilesWithAbsolutePathListaExtensiones(input_folder)), 2], dtype=object)

    idx = 0

    outputfolder = os.path.abspath(output_folder)

    for animal in os.listdir(input_folder):
        idx_aux = 0

        animal_imgs = getFilesWithAbsolutePathListaExtensiones(input_folder + animal)
        animal_imgs.sort(key=natural_keys)

        output_folder = outputfolder + os.sep + animal + os.sep

        for img in animal_imgs:
            # origNameFile = img.split(animal)[-1]
            # origNameFile = origNameFile.replace(os.sep,"")

            # Lee la imagen original y la reescala
            imgResized = resize_img(img)
            if imgResized == 0:
                continue

            # New output name:

            new_name = idx_aux

            # Save resized image in output
            fileOutput = output_folder + str(new_name) + ".jpg"

            try:
                # save img
                imgResized.save(fileOutput)
            except:
                print('Error Saving Resized image:', fileOutput)   #XXX fileinput
                continue

            idx_aux += 1

        for img in animal_imgs:
            csvArray[idx,:] = [img, animal]
            idx += 1

In [35]:
# resize images and save them in output folder
save_resized_imgs(INPUT_FOLDER, OUTPUT_FOLDER)

C:\Users\albam\AppData\Local\Temp\ipykernel_11140\3645770621.py:37: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((basewidth,baseheight), Image.ANTIALIAS) # cambiar pilImage.ANTIALIAS a pilImage.Resampling.LANCZOS


In [36]:
# create csv for the inputfolder images (originals)
create_CSV(INPUT_FOLDER, "_originals")
# create csv for the output images (resized)
create_CSV(OUTPUT_FOLDER,"_resized")


CSV salvado:  ..\csv\imgs_and_labels_list_originals.csv

CSV salvado:  ..\csv\imgs_and_labels_list_resized.csv


In [47]:
def load_and_save_data_from_dir(input_folder = INPUT_FOLDER):
    input_folder = os.path.abspath(input_folder) + os.sep
    print(input_folder)
    npArray = np.empty([len(getFilesWithAbsolutePathListaExtensiones(INPUT_FOLDER)), 2], dtype=object)

    idx_aux = 0

    for animal in os.listdir(input_folder):
        print("\nStarting to process " + animal)
        animal_imgs = getFilesWithAbsolutePathListaExtensiones(input_folder + animal)
        animal_imgs.sort(key=natural_keys)
        for img in animal_imgs:
            img=cv2.imread(img)
            # convert image to array
            img = np.array(img)
            img_from_ar = Image.fromarray(img, 'RGB')

            # image of same size needed for CNN, ensure they are -> 50X50
            resized_image = img_from_ar.resize((250, 250))  # tam = cte, move up, bool to change size (if)

            npArray[idx_aux,:] = [resized_image, animal]
            idx_aux += 1

    np.save("../np_arrays/imgs_animals",npArray)

    print("Finished")

In [48]:
load_and_save_data_from_dir(OUTPUT_FOLDER)

C:\Users\albam\OneDrive - UNIVERSIDAD DE HUELVA\2. Universidad\Proyects\2. Animal Recognition\datasets\animals\data_resized\

Starting to process Bear

Starting to process Chicken

Starting to process Turtle
Finished


In [50]:
imgs_animals = np.load("../np_arrays/imgs_animals.npy", allow_pickle=True)

#### For the moment we have:
- Resized images in folder
- CSV with resized images relative path + label
- CSV with original images relative path + label
- NP array with images + label